### 2018/2019 - Task List 10

1. Implement Naive Bayes classifier with pyro
    - create apropriate parameters (mean and std for a and b, sigma - noise)
    - provide optimization procedure
    - check appropriateness of implemented method with selected dataset


# Required imports

In [1]:
%matplotlib inline
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive

In [2]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)

In [3]:
from sklearn import datasets, model_selection
from sklearn.naive_bayes import GaussianNB


## Solutions

### sklearn solution

In [4]:
iris = datasets.load_iris()

gnb = GaussianNB()


X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(
    iris.data, iris.target, test_size=0.33, random_state=5)

In [5]:
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)

In [6]:
kfold = 5
scoring = ['accuracy', 'f1_micro', 'f1_macro']
cv_results = model_selection.cross_validate(gnb, X_train, Y_train, cv=kfold, scoring=scoring)

#displaying the mean and standard deviation of the prediction
for score in scoring: 
    msg = "%s: %f (%f)" % ('NB ' + score, cv_results['test_' + score].mean(), cv_results['test_' + score].std())
    print(msg)

NB accuracy: 0.960902 (0.035958)
NB f1_micro: 0.960902 (0.035958)
NB f1_macro: 0.960191 (0.036241)


### self-made NB

In [7]:
class NaiveBayesClassifier:
    
    def __init__(self, x_data, y_data):
        self.X = x_data
        self.y = y_data
        self.attributes_number = x_data.shape[1]
        self.classes = np.unique(y_data)
    
    def fit(self):
        pass
    
    def predict(self):
        pass
    
    def div_by_category(self, x_data, y_data):
        X_cl = {}

        for i in range(len(x_data)):
            if y_data[i] not in X_cl.keys():
                X_cl[y_data[i]] = list()
            X_cl[y_data[i]].append(x_data[i])

        for cl_key in categories:
            X_cl[cl_key] = np.array(X_cl[cl_key])

        return X_cl
    
    def model(self, x_data, label):
        mean = torch.tensor(np.random.choice(x_data, 1))
        scale = torch.tensor(1.0)
        with pyro.plate('data_loop', len(x_data)):
            pyro.sample('prob', dist.Normal(mean, scale), obs=x_data, infer={'is_auxiliary': True})
        
    def guide(self, x_data, label):
        mean = pyro.param('mean' + str(label), torch.tensor(np.random.choice(x_data, 1)))
        scale = pyro.param('scale' + str(label), torch.tensor(np.random.choice(1.)), constraint=constraints.positive)
        pyro.sample('prob', dist.Normal(mean, scale))
        
        

In [25]:
def div_by_category(x_data, y_data):
    X_cl = {}

    for i in range(len(x_data)):
        for attribute in range(len(x_data[0])):
            if (y_data[i], attribute) not in X_cl.keys():
                X_cl[(y_data[i], attribute)] = list()
            X_cl[(y_data[i], attribute)].append(x_data[i][attribute])

    for cl_key in np.unique(y_data):
        for attribute in range(len(x_data[0])):
            X_cl[cl_key, attribute] = np.array(X_cl[cl_key, attribute])

    return X_cl

def model(x_data, label):
    #mean = torch.tensor(np.random.choice(x_data, 1)).double()
    mean = torch.tensor(2.0).double()
    scale = torch.tensor(1.0).double()
    with pyro.plate('data_loop' + str(label), len(x_data)):
        pyro.sample('prob' + str(label), dist.Normal(mean, scale), obs=x_data)

def guide(x_data, label):
#     x_data = torch.from_numpy(x_data).double()
    mean = pyro.param('mean' + str(label), torch.tensor(2.0))
    scale = pyro.param('scale' + str(label), torch.tensor(1.0), constraint=constraints.positive)
    pyro.sample('prob' + str(label), dist.Normal(mean.double(), scale.double()), infer={'is_auxiliary': True})
    
def train(data, label, num_steps=3000):
    pyro.clear_param_store()
    
    optim = pyro.optim.Adam({"lr": 0.045})
    svi = pyro.infer.SVI(model=model,
                         guide=guide,
                         optim=optim,
                         loss=pyro.infer.Trace_ELBO(), num_samples=len(data))

    losses = []
    t = tqdm(range(num_steps))
    for j in t:
        loss = svi.step(torch.from_numpy(data), label)
        losses.append(loss)
        print(pyro.param("mean" + str(label)))
        t.set_postfix(loss=loss)
    return pyro.param("mean" + str(label)), pyro.param("scale" + str(label)), losses

In [26]:
divided_data = div_by_category(X_train, Y_train)
train(divided_data[(0, 0)], (0,0))








  0%|          | 0/3000 [00:00<?, ?it/s]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=185]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=185]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=185]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 0/3000 [00:00<?, ?it/s, loss=184]






  0%|          | 14/3000 [00:00<00:22, 131.85it/s, loss=184]






  0%|          | 14/3000 [00:00<00:22, 131.85it/s, loss=184]






  0%|         

tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)









  0%|          | 14/3000 [00:00<00:22, 131.85it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=182]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=182]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss=183]






  1%|          | 26/3000 [00:00<00:23, 126.79it/s, loss

tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)


  1%|▏         | 40/3000 [00:00<00:23, 128.50it/s, loss=182]






  1%|▏         | 40/3000 [00:00<00:23, 128.50it/s, loss=181]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=181]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=182]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=181]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=182]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=179]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=182]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=180]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=181]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=182]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=182]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=182]






  2%|▏         | 55/3000 [00:00<00:22, 133.65it/s, loss=179]






  2%|▏         | 66/3000 [00:00<00:24, 121.49it/s, loss=179]



...

tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)


 65%|██████▌   | 1952/3000 [00:15<00:09, 106.58it/s, loss=96]  






 65%|██████▌   | 1952/3000 [00:15<00:09, 106.58it/s, loss=96.4]






 65%|██████▌   | 1952/3000 [00:15<00:09, 106.58it/s, loss=95.1]






 65%|██████▌   | 1952/3000 [00:15<00:09, 106.58it/s, loss=95.6]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=95.6]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=96.4]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=95.9]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=95]  






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=96]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=96]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=96.1]






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=96]  






 65%|██████▌   | 1964/3000 [00:15<00:09, 105.89it/s, loss=95.8]/home/marcin/anaconda3/lib/python3.7/site-packages/pyro/poutine/trace_struct.py:259: UserWarning: Enc

tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(2., requires_grad=True)
tensor(nan, requires_grad=True)


ValueError: The parameter loc has invalid values
Trace Shapes:
 Param Sites:
   mean(0, 0)
  scale(0, 0)
Sample Sites:








 65%|██████▌   | 1964/3000 [00:34<00:09, 105.89it/s, loss=nan]

In [10]:
divided_data = div_by_category(X_train, Y_train)

adam_params = {"lr": 0.5}
optimizer = pyro.optim.Adam(adam_params)
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

n_steps = 2501
for step in range(n_steps):
    svi.step(divided_data[(2, 2)], (2,2))
    if step % 100 == 0:
        print_progress()


ValueError: Error while computing log_prob at site 'prob(2, 2)':
The value argument to log_prob must be a Tensor
  Trace Shapes:     
   Param Sites:     
  Sample Sites:     
prob(2, 2) dist 33 |
          value 33 |

In [ ]:
pyro.clear_param_store()
type("ad")

In [ ]:
for i in range(1, 10):
    print(pyro.sample('test', dist.Normal(torch.zeros(1, 1), 1.).independent(1)))

In [ ]:
for i in range(1, 10):
    print(pyro.sample('test', dist.Normal(torch.zeros(1, 1), 1.).to_event(1)))